In [1]:
import nocd
from nocd.utils import *
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
from nocd.nn import models_deep
from nocd.utils import to_sparse_tensor
from cdlib import evaluation
from sklearn.preprocessing import normalize
import networkx as nx
from scipy.sparse import csr_matrix
%matplotlib inline

torch.set_default_tensor_type(torch.cuda.FloatTensor)

c:\Python311\Lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ..\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


Generate A

In [2]:
edges = np.loadtxt('./dataset/ogbn_proteins/raw/edge.csv.gz', delimiter=',', dtype=int)
num_nodes = max(edges.max(), edges.min()) + 1
adjacency_matrix = csr_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), shape=(num_nodes, num_nodes))

Generate X

In [3]:
node_labels = np.loadtxt('./dataset/ogbn_proteins/raw/node-label.csv.gz', delimiter=',', dtype=int)
num_rows = node_labels.shape[0]
num_columns = node_labels.shape[1]
label_matrix = csr_matrix((np.ones(num_rows), (node_labels[:, 0], node_labels[:, 1])), shape=(num_rows, num_columns))

Generate Z_dt

In [4]:
species_data = np.loadtxt('./dataset/ogbn_proteins/raw/node_species.csv.gz', delimiter=',', dtype=str)
unique_species = np.unique(species_data)
unique_protein_ids = np.arange(len(species_data))
affiliation_matrix = np.zeros((len(unique_protein_ids), len(unique_species)), dtype=int)
for i, protein_id in enumerate(unique_protein_ids):
    species = species_data[i]
    j = np.where(unique_species == species)[0][0]
    affiliation_matrix[i, j] = 1

Generate orG

In [5]:
import pandas as pd
df1=pd.read_csv("./dataset/ogbn_proteins/raw/edge.csv.gz")
G = nx.from_pandas_edgelist(df1, "1", "96401")

In [6]:
A=adjacency_matrix
X=label_matrix
Z_gt=affiliation_matrix
orG = G
N, K = Z_gt.shape

In [7]:
def mod_overlap(Z, G_w, thresh=0.5):
    from cdlib import NodeClustering
    Z_pred = Z.cpu().detach().numpy() > thresh
    communities = coms_matrix_to_list(Z_pred)
    new_com = []
    for ls in communities:
        if(len(ls)==0):
            continue
        new_com.append(ls)
    coms = NodeClustering(new_com, G_w)
    mod = evaluation.modularity_overlap(G_w, coms)
    return mod.score

In [8]:
'''
We generate hybrid graphs which real world graphs with imputated weight. 
Weights are imputation based on the assumption that 
To nodes sharing more communities would have higher weight. 
Base Idea: for each edge if two nodes share a community add 1 to the weight:
'''
# A_w = nx.to_scipy_sparse_matrix(G_w,format='csr')
# G_w =  nx.from_numpy_matrix(A_w.toarray(), create_using=nx.Graph)


def get_num_shared_community(u,v, Z, K):
    count = 0
    for k in range(K):
        if(Z[u][k]>0 and Z[v][k]>0):
            count +=1
            
    return count


def weight_imputation(A, Z_gt):
    Gw = G
    Z = Z_gt
    N, K = Z_gt.shape
    
    edges = Gw.edges()
    
    count = 0
    for pair in edges:
        u, v = pair
        
        Gw[v][u]['weight']=Gw[u][v]['weight'] = int(get_num_shared_community(u,v, Z, K ) + 1)
        if(Gw[v][u]['weight']==1):
            count+=1
            
    print("non sharing edges: ", count)
    print("in total edges: ", Gw.number_of_edges())
        
    
    return Gw
        
        
        
        
        
Gw = weight_imputation(A, Z_gt) 
# orG = Gw
Aw = nx.to_scipy_sparse_matrix(Gw,format='csr')
# print(Aw[6])
    
    

non sharing edges:  7676421
in total edges:  39561251


MemoryError: 

In [14]:
Gw[1][1]

KeyError: 1

In [12]:
G.number_of_edges()

39561251